In [5]:
from transformers import BertTokenizer,AlbertTokenizer,AutoTokenizer, AutoModelForQuestionAnswering ,BertForQuestionAnswering, AlbertForQuestionAnswering
import torch

'''
# old BERT model (no longer used):
model_name='bert-large-uncased-whole-word-masking-finetuned-squad'
model = BertForQuestionAnswering.from_pretrained(model_name)
'''

"\n# old BERT model (no longer used):\nmodel_name='bert-large-uncased-whole-word-masking-finetuned-squad'\nmodel = BertForQuestionAnswering.from_pretrained(model_name)\n"

In [6]:
# tokenizer = BertTokenizer.from_pretrained(model_name)

example = {"key": 6, "transcript": "Heading is one seven five, target is black, blue, and grey fighter jet, tool to deploy is machine gun.", "tool": "machine gun", "heading": "175", "target": "black, blue, and grey fighter jet"}

answer_text = example["transcript"]
questions = {
    'heading': "What is the heading value?", 
    'target': "What is the full target description?",
    'tool': "What is the tool to deploy?"
}

In [6]:
import re

number_map = {
    "zero": "0",
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "niner": "9"
}

questions = {
    'heading': "What is the heading value?", 
    'target': "What is the full target description?",
    'tool': "What is the tool to deploy?"
}

def qa(questions,answer_text,model,tokenizer):
    answers = {}
    for key, question in questions.items():
      inputs = tokenizer.encode_plus(question, answer_text, add_special_tokens=True, return_tensors="pt")
      # the loop below moves the input tensors to the same device as the model (gpu)
      #for k in inputs:
        #inputs[k] = inputs[k].to(device)
      inputs = inputs.to(device)
      input_ids = inputs["input_ids"].tolist()[0]

      text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
      #print(text_tokens)
      outputs = model(**inputs)
      answer_start_scores=outputs.start_logits
      answer_end_scores=outputs.end_logits

      answer_start = torch.argmax(
          answer_start_scores
      )  # Get the most likely beginning of answer with the argmax of the score
      answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
      answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

      # Combine the tokens in the answer and print it out.""
      answers[key] = answer.replace("#","")

      #print(f'{key}: "' + answers[question] + '"')
    
    # convert words to numbers
    answers['heading'] = "".join(number_map[word] for word in answers['heading'].split())
    
    # cleaning up the output
    answers['tool'] = re.sub(r'\s*-\s*', '-', answers['tool'])
    answers['target'] = re.sub(r'\s+,', ',', answers['target'])
    if answers['tool'] == 'emp':
        answers['tool'] = answers['tool'].upper()
    
    return answers

In [8]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# model: ELECTRA (Efficiently Learning an Encoder that Classifies Token Replacements Accurately):
tokenizer = AutoTokenizer.from_pretrained("valhalla/electra-base-discriminator-finetuned_squadv1")
model = AutoModelForQuestionAnswering.from_pretrained("valhalla/electra-base-discriminator-finetuned_squadv1")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # runs model on gpu




ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerN

In [9]:
# TESTING

from pathlib import Path
import jsonlines
import json
import math
import time

current_directory = Path.cwd()
file_path = current_directory / '..' / '..' / 'novice' / 'nlp.jsonl'
nlp_file = file_path.resolve()
print(f"nlp_file: {nlp_file}\ncurrent_directory: {current_directory}")

transcripts = []
extracted_commands = []

with open(nlp_file, 'r') as file:
    for line in file:
        data = json.loads(line)
        transcripts.append(data["transcript"])
        extracted_commands.append({"heading": data["heading"], "target": data["target"], "tool": data["tool"]})
        
accuracy_count = 0
length = len(transcripts)

start_time = time.time()
           
for i in range(length):
    json_output = qa(questions, transcripts[i], model, tokenizer)
    #if i% math.ceil(length/100) == 0:
    if json_output != extracted_commands[i]:
        print(f"Iteration: {i}/{length}")
        print(f"json_output: {json_output}")
        print(f"extracted_commands[i]: {extracted_commands[i]}")
    if json_output == extracted_commands[i]:
        accuracy_count += 1
        
end_time = time.time()
                                                          
accuracy = accuracy_count / length * 100
print(f"Accuracy: {accuracy:.02f}%")

time_taken = end_time - start_time
print(f"Time taken: {time_taken}")

nlp_file: /home/jupyter/novice/nlp.jsonl
current_directory: /home/jupyter/til-24-base/nlp
Accuracy: 100.00%
Time taken: 118.0271303653717
